In [2]:
import gensim.downloader as api

#Download the model
word2vec_model = api.load("word2vec-google-news-300")

In [70]:
# Verifier la presence de mon nom /  Verify the presence of my name
prenom = "Gaston"
if prenom in word2vec_model:
    print(f"Le prénom {prenom} est représenté par un vecteur.")
    print(f"The first name {prenom} is represented by a vector.")
else:
    print(f"Le prénom {prenom} n'est pas représenté par un vecteur.")
    print(f"The first name {prenom} is not represented by a vector.")


Le prénom Gaston est représenté par un vecteur.
The first name Gaston is represented by a vector.


In [19]:
#Le mot le plus similaire a mon nom / The word most similar to my name
word2vec_model.most_similar('Gaston', topn=3)[0][0]

'Lenoir'

In [9]:
#Similarite entre Computer et Software / Similarite entre Computer et Software
word2vec_model.similarity('Computer','Software')

0.46974862

In [10]:
# correspondance plus forte / strongest correspondence
word2vec_model.most_similar('Computer', topn=5)

[('Computers', 0.70357745885849),
 ('computer', 0.647333562374115),
 ('Computing', 0.5785260796546936),
 ('Enterprises_GoGrid_Gomez', 0.5636221170425415),
 ('computers', 0.5459214448928833)]

In [12]:
#Trouver l'Intru / Find the Intruder
word2vec_model.doesnt_match('computer software intelligence engineering'.split())

'intelligence'

In [33]:
#Operations Semantiques / Semantic Operations
print(word2vec_model.most_similar(positive=["India", "currency"], negative=["country"], topn=1)[0][0])
print(word2vec_model.most_similar(positive=["buy", "past"], negative=["present"], topn=1)[0][0])
print(word2vec_model.most_similar(positive=["Chile", "capital"], negative=["country"], topn=1)[0][0])
print(word2vec_model.most_similar(positive=["China", "president", "name"], negative=["country","president"], topn=1)[0][0])
print(word2vec_model.most_similar(positive=["Horse","female"], negative=["male"], topn=1)[0][0])

rupee
bought
Chilean
names
Horses


## Entraîner notre propre modèle depuis la page Wikipédia « computer science ». / Train our own model from the "computer science" Wikipedia page.

In [36]:
import bs4 as bs
import urllib.request
import re
import nltk

scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Computer')
wiki_info = scrapped_data.read()
print(wiki_info)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Computer - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-

In [38]:
# Parser ce texte xml avec : bs.BeautifulSoup(wiki_info,'html.parser') / Parser this xml text with: bs.BeautifulSoup(wiki_info,'html.parser')
soup = bs.BeautifulSoup(wiki_info,'html.parser')

In [39]:
#Utiliser la bonne balise pour extraire les paragraphes / Use the right tag to extract paragraphs
paragraphs = soup.find_all('p')

In [41]:
#Extraire le texte brut et comparer avec le texte sur la page Wikipédia / Extract the plain text and compare with the text on the Wikipedia page 
text = ""
for para in paragraphs:
    text += para.text

print(text)



A computer is a machine that can be programmed to automatically carry out sequences of arithmetic or logical operations (computation). Modern digital electronic computers can perform generic sets of operations known as programs. These programs enable computers to perform a wide range of tasks. The term computer system may refer to a nominally complete computer that includes the hardware, operating system, software, and peripheral equipment needed and used for full operation; or to a group of computers that are linked and function together, such as a computer network or computer cluster.
A broad range of industrial and consumer products use computers as control systems, including simple special-purpose devices like microwave ovens and remote controls, and factory devices like industrial robots. Computers are at the core of general-purpose devices such as personal computers and mobile devices such as smartphones. Computers power the Internet, which links billions of computers and users.

In [42]:
#Segmenter le texte en phrases puis en mots / Segment text into sentences and then into words
from nltk.tokenize import sent_tokenize, word_tokenize

# Segmenter le texte en phrases / Segment text into sentences
sentences = sent_tokenize(text)

# Segmenter les phrases en mots / Segment sentences into words
words = [word_tokenize(sentence) for sentence in sentences]


In [45]:
# Enlever les stopwords / Remove stopwords
from nltk.corpus import stopwords

# Télécharger les stop words si nécessaire / Download stopwords if necessary
nltk.download('stopwords')
nltk.download('punkt')

# Liste des stop words en anglais / List of English stop words
stop_words = set(stopwords.words('english'))

# Enlever les stop words des mots segmentés / Remove stop words from segmented words
filtered_words = [[word for word in word_list if word.lower() not in stop_words and word.isalnum()] for word_list in words]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
#Afficher les mots non porteurs / Display non-bearing words
print(stop_words)

{'at', 'y', 'herself', 'than', 'do', 'this', 'with', 'and', 'further', 'the', 'their', 'ours', 'what', 'ain', "couldn't", 'here', 'himself', 'to', 'more', 'of', 'for', 'if', 'above', 'are', 'just', 'very', 'through', 'from', 'too', 'down', 'o', 'his', 'we', 'had', 'mightn', 'yours', 'ma', 'was', 'be', 'haven', 'needn', "wouldn't", 'couldn', 'over', 'shouldn', 'can', "it's", 'you', 'below', "isn't", 'don', 'should', "you'd", 'then', "should've", 'aren', "hadn't", 'mustn', "mustn't", 'her', 's', 'itself', 're', 'by', 'such', 'hasn', 'how', 'd', "you're", 'only', 'there', 'doesn', 'myself', 'up', "weren't", "wasn't", 'll', "didn't", 'who', "shan't", 'against', 'hadn', 'each', "won't", 'in', 'its', 'been', 'has', 'or', 'as', 'them', 'nor', 'him', "haven't", 've', "you've", 'now', 'why', 'own', 'ourselves', 'is', 'not', 'until', 'it', 'm', 'he', "mightn't", 'about', 'these', 'me', 'have', 'having', 'didn', 'theirs', 'your', 'those', 'when', "aren't", "doesn't", 'hers', "hasn't", "you'll", '

In [55]:
# Entraîner le modèle Word2Vec / Training the Word2Vec model
from gensim.models import Word2Vec

model = Word2Vec(filtered_words, vector_size=300, window=5, min_count=2, workers=4)



In [58]:
#Top 5 des mots similaires a computer / Top 5 words similar to computer
model.wv.most_similar("computer", topn=5)

[('different', 0.424376517534256),
 ('computers', 0.3835306465625763),
 ('memory', 0.3830599784851074),
 ('first', 0.3739147484302521),
 ('used', 0.3384435474872589)]

In [59]:
#Vérifier la similarité entre "computer" et "software" / Check the similarity between "computer" and "software".
model.wv.similarity("computer", "software")

0.1368767

Résultats
Modèle pré-entraîné (word2vec-google-news-300) : La similarité entre "Computer" et "Software" est de 0.46974862.
Modèle entraîné localement : La similarité entre "computer" et "software" est de 0.1368767.
La différence de similarité observée est normale et attendue en raison des différences de taille et de qualité du corpus d'entraînement.

Results
Pre-trained model (word2vec-google-news-300): The similarity between "Computer" and "Software" is 0.46974862.
Locally trained model: The similarity between "computer" and "software" is 0.1368767.
The observed difference in similarity is normal and expected given the differences in size and quality of the training corpus.

## Analyse de sentiments / Sentiment analysis

In [61]:
import nltk

# Télécharger le lexique vader_lexicon / Download the vader_lexicon
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...


True

In [67]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores('comfortable')

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.5106}

In [68]:
# programme qui permet de représenter les phrases suivantes par des vecteurs / Program to represent the following sentences as vectors
# Liste des phrases à analyser / List of sentences to be analyzed
phrases = [
    "The atmosphere and staff were very friendly",
    "I wanted to order room service however they won’t offering the service in the morning",
    "The temperature of the room was hard to regulate",
    "Great hotel but unfortunate receptionist",
    "It was alright but could have been better",
    "I liked the layout of the room"
]

# Fonction pour obtenir le score de polarité d'une phrase / Function to get the polarity score of a sentence
def get_polarity_scores(sentence):
    return sia.polarity_scores(sentence)

# Analyser les phrases et représenter par des vecteurs de polarité / Analyze sentences and represent with polarity vectors
polarities = [get_polarity_scores(phrase) for phrase in phrases]

# Afficher les résultats / Show results
for phrase, polarity in zip(phrases, polarities):
    print(f"Phrase / Sentences: {phrase}")
    print(f"Polarité / Polarity: {polarity}")
    print()

Phrase: The atmosphere and staff were very friendly
Polarité: {'neg': 0.0, 'neu': 0.632, 'pos': 0.368, 'compound': 0.5413}

Phrase: I wanted to order room service however they won’t offering the service in the morning
Polarité: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Phrase: The temperature of the room was hard to regulate
Polarité: {'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'compound': -0.1027}

Phrase: Great hotel but unfortunate receptionist
Polarité: {'neg': 0.419, 'neu': 0.314, 'pos': 0.267, 'compound': -0.3506}

Phrase: It was alright but could have been better
Polarité: {'neg': 0.0, 'neu': 0.529, 'pos': 0.471, 'compound': 0.6542}

Phrase: I liked the layout of the room
Polarité: {'neg': 0.0, 'neu': 0.641, 'pos': 0.359, 'compound': 0.4215}



In [69]:
# Fonction de classification des commentaires / Function to classify comments
# Fonction pour classifier les commentaires en positif et négatif / Function to classify comments as positive or negative
def classify_comments(phrase):
    scores = get_polarity_scores(phrase)
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Classifier les phrases / Classifying sentences
classifications = [classify_comments(phrase) for phrase in phrases]

# Afficher les classifications / Display classifications
for phrase, classification in zip(phrases, classifications):
    print(f"Phrase: {phrase}")
    print(f"Classification: {classification}")
    print()

Phrase: The atmosphere and staff were very friendly
Classification: Positive

Phrase: I wanted to order room service however they won’t offering the service in the morning
Classification: Neutral

Phrase: The temperature of the room was hard to regulate
Classification: Negative

Phrase: Great hotel but unfortunate receptionist
Classification: Negative

Phrase: It was alright but could have been better
Classification: Positive

Phrase: I liked the layout of the room
Classification: Positive

